In [3]:
import os
import sys
import time
import numpy as np
import laspy
import xarray as xr
import cartopy.crs as ccrs
import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset
from pyproj import CRS
from osgeo import gdal
import netCDF4 as nc
from laspy.file import File
from laspy.header import LasHeader
# sys.path.append("..")
from parser.parser_3dtiles.parser_3dtiles import Parser3DTiles
from tools.query import Query

## 1 ten thousand facts per addition

In [3]:
p3d = Parser3DTiles()
testPath = "/home/ydy/3dsim/DA1_3D_Buildings/DA1_3D_Buildings_Merged.json"
p3d.add_data(testPath,featureType='Building',createTime='20230116', validTime=['19950923', '20050101'])
p3d1 = Parser3DTiles()
testPath1 = "/home/ydy/3dsim/DA6-10_3D_Buildings/tileset.json"
p3d1.add_data(testPath1,featureType='Building',createTime='20230116', validTime=['19950923', '20050101'])
p3d2 = Parser3DTiles()
testPath2 = "/home/ydy/3dsim/DA20_3D_Buildings/tileset.json"
p3d2.add_data(testPath2,featureType='Building',createTime='20230116', validTime=['19950923', '20050101'])
p3d3 = Parser3DTiles()
testPath3 = "/home/ydy/3dsim/DA5_3D_Buildings/tileset.json"
p3d3.add_data(testPath3,featureType='Building',createTime='20230116', validTime=['19950923', '20050101'])
p3d4 = Parser3DTiles()
testPath4 = "/home/ydy/3dsim/DA16_3D_Buildings/tileset.json"
p3d4.add_data(testPath4,featureType='Building',createTime='20230116', validTime=['19950923', '20050101'])
p3d5 = Parser3DTiles()
testPath5 = "/home/ydy/3dsim/DA5_3D_Buildings/tileset.json"
p3d5.add_data(testPath5,featureType='Building',createTime='20230116', validTime=['19950923', '20050101'])

## 2 20 queries per performance test

In [4]:
for i in range(20):
    ix = 4
    query  = Query()
    result = query.query_rootSceneAsset()
    print("size of root scene: ", len(result))
    # print(result[ix])
    p3d = Parser3DTiles()
    start_time = time.time()
    p3d.save_data_to3dtiles(sceneAsset=result[ix], path='./tileset.json',query=query)
    end_time = time.time()
    execution_time_ms = (end_time - start_time) * 1000
    print(f"代码执行时间：{execution_time_ms:.2f}毫秒")

## 3 三维点云解析

In [6]:
las = laspy.read("/home/program/3dsim/data/pointcloud/building_01.las")

### 3.1 点云坐标变换

In [5]:
# 读取Laz点云文件
input_laz_file = "/home/program/3dsim/data/pointcloud/building_01.las"
output_laz_file = "/home/program/3dsim/data/pointcloud/output.las"

# 读取点云数据
las = laspy.read(input_laz_file)

# 获取点云数据
xyz = np.vstack((las.x, las.y, las.z)).T
rgb = np.vstack((las.red, las.green, las.blue)).T


"""
@description: 这段代码定义了一个名为 `transform2world` 的变换矩阵，并将其应用到点云数据中。变换矩阵是一个 4x4 的矩阵，用于将点云从局部坐标系转换到世界坐标系。

具体来说，这个变换矩阵将点云的 X、Y、Z 坐标分别乘以一个缩放因子 `0.000008998719243599958`,
然后将点云沿 X 轴平移 `103.778333`，沿 Y 轴平移 `1.284444`。     
最后，变换矩阵的第四行和第四列都是 `[0, 0, 0, 1]`，表示不对点云进行旋转和投影变换。
"""
# 定义变换矩阵
transform2world = np.array([
    [0.000008998719243599958, 0.0, 0.0, 103.778333],
    [0.0, 0.000008998719243599958, 0.0, 1.284444],
    [0.0, 0.0, 1.0, 0.0],
    [0.0, 0.0, 0.0, 1.0]
])

# 应用变换矩阵到点云数据
transformed_xyz = np.dot(xyz, transform2world[:3, :3].T) + transform2world[:3, 3]

# 创建新的Laz文件并保存点云
header = LasHeader(point_format=2, version="1.2")

# TODO laspy 2.0
#header = laspy.create(point_format=2, file_version='1.2')


header.offset = [0.0, 0.0, 0.0]  # 根据实际情况设置正确的偏移
header.scale = [0.01, 0.01, 0.01]  # 根据实际情况设置正确的比例


out_las = File(output_laz_file, mode="w", header=header)
# TODO laspy 2.0
#las.write(output_laz_file)

out_las.x = transformed_xyz[:, 0]
out_las.y = transformed_xyz[:, 1]
out_las.z = transformed_xyz[:, 2]
out_las.red = rgb[:, 0]
out_las.green = rgb[:, 1]
out_las.blue = rgb[:, 2]

# 关闭输入和输出laz文件
las.close()
out_las.close()

## 4 三维物理场数据解析

In [34]:
def parser_physicalfield_v1():
    data=xr.open_dataset('/home/program/3dsim/data/physicalfield/example_1.nc')
    print("\nnc文件的概况:")
    print(data)
    print("\nnc文件中的变量:")
    lat_values = data.variables.keys()
    print(lat_values)

def parser_physicalfield_v2():
    nc_dataset = Dataset('/home/program/3dsim/data/physicalfield/example_1.nc', 'r')
    print("\nnc版本:")
    print(nc_dataset.data_model)
    print("\nnc文件中包含的变量:")
    print(nc_dataset.variables.keys())
    print("\nnc文件中包含的属性:")
    print(nc_dataset.ncattrs())

    lon = nc_dataset['longitude'][:]
    lat = nc_dataset['latitude'][:]
    x_min, x_max = lon.min(), lon.max()
    y_min, y_max = lat.min(), lat.max()


parser_physicalfield_v1()


nc文件的概况:
<xarray.Dataset> Size: 1kB
Dimensions:  (time: 1, level: 4, lat: 5, lon: 10)
Coordinates:
  * lat      (lat) int32 20B 20 30 40 50 60
  * lon      (lon) int32 40B -160 -140 -118 -96 -84 -52 -45 -35 -25 -15
  * level    (level) int32 16B 1000 850 700 500
  * time     (time) datetime64[ns] 8B 1996-01-01T12:00:00
Data variables:
    temp     (time, level, lat, lon) float32 800B ...
    rh       (time, lat, lon) float32 200B ...
Attributes:
    source:   Fictional Model Output

nc文件中的变量:
KeysView(Frozen({'temp': <xarray.Variable (time: 1, level: 4, lat: 5, lon: 10)> Size: 800B
[200 values with dtype=float32]
Attributes:
    long_name:  temperature
    units:      celsius, 'rh': <xarray.Variable (time: 1, lat: 5, lon: 10)> Size: 200B
[50 values with dtype=float32]
Attributes:
    long_name:    relative humidity
    valid_range:  [0. 1.], 'lat': <xarray.IndexVariable 'lat' (lat: 5)> Size: 20B
array([20, 30, 40, 50, 60], dtype=int32)
Attributes:
    units:    degrees_north, 'lon': <

### 4.1 地理坐标系下的ERA5再分析数据

In [14]:
in_path = r"/home/program/3dsim/data/physicalfield/test1.nc"
out_path = r"/home/program/3dsim/data/physicalfield/test1.tif"

# 获取分辨率和左上角像素坐标值
rootgrp = nc.Dataset(in_path)
lon = rootgrp['longitude'][...].data
lat = rootgrp['latitude'][...].data
x_min, y_max = lon.min(), lat.max()
x_res, y_res = abs(float(lon[1]-lon[0])), abs(float(lat[1]-lat[0]))

# 仿射变换六参数
gt = (x_min, x_res, 0, y_max, 0, -y_res)

# 获取地理坐标系
crs = CRS.from_epsg(4326)
wkt = crs.to_wkt()

# 读取数据
r = rootgrp['r']
nodata = int(r._FillValue)
r = r[...].filled(nodata)

# 计算月平均相对湿度
r = r.mean(axis=0).astype(np.int32)

# 创建GeoTIFF文件并写入数据
driver = gdal.GetDriverByName('GTiff')
ds = driver.Create(out_path, r.shape[1], r.shape[0], 1, 
                gdal.GDT_Int32, options=["COMPRESS=LZW"])
ds.SetGeoTransform(gt)
ds.SetProjection(wkt)
band = ds.GetRasterBand(1)
band.WriteArray(r)
band.SetNoDataValue(nodata)

ds = band = None

### 4.2 投影坐标系下的反演数据

In [5]:
in_path = r"/home/program/3dsim/data/physicalfield/test2.nc"
out_path = r"/home/program/3dsim/data/physicalfield/test2.tif"

# 获取分辨率和左上角像素坐标值
rootgrp = nc.Dataset(in_path)
lon = rootgrp['x'][...].data
lat = rootgrp['y'][...].data
x_min, y_max = lon.min(), lat.max()
x_res, y_res = abs(float(lon[1]-lon[0])), abs(float(lat[1]-lat[0]))

# 仿射变换六参数
gt = (x_min, x_res, 0, y_max, 0, -y_res)

# 获取地理坐标系
gr = rootgrp['polar_stereographic']
wkt = gr.crs_wkt

# 读取数据
PFR = rootgrp['PFR']
nodata = int(PFR._FillValue)
PFR = PFR[0, ...].filled(nodata)

# 创建GeoTIFF文件并写入数据
driver = gdal.GetDriverByName('GTiff')
ds = driver.Create(out_path, PFR.shape[1], PFR.shape[0], 1, 
                gdal.GDT_Int16, options=["COMPRESS=LZW"])
ds.SetGeoTransform(gt)
ds.SetProjection(wkt)
band = ds.GetRasterBand(1)
band.WriteArray(PFR)
band.SetNoDataValue(nodata)

ds = band = None